In [1]:
from langchain import hub
from langchain.chains import SequentialChain

In [2]:
import os
import yaml

config = yaml.safe_load(open("config.yml"))

os.environ["OPENAI_API_KEY"] = config["OPEN_AI_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
os.environ["WANDB_PROJECT"] = config["WANDB_PROJECT"]

# Tree of Thoughts

In [3]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
#from langchain_openai import ChatOpenAI

In [4]:
from langchain import hub

cot_step1 = hub.pull("rachnogstyle/nlw_jan24_cot_step1")
cot_step2 = hub.pull("rachnogstyle/nlw_jan24_cot_step2")
cot_step3 = hub.pull("rachnogstyle/nlw_jan24_cot_step3")
cot_step4 = hub.pull("rachnogstyle/nlw_jan24_cot_step4")

In [5]:
model = "gpt-3.5-turbo"

chain1 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step1,
    output_key="solutions"
)

chain2 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step2,
    output_key="review"
)

chain3 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step3,
    output_key="deepen_thought_process"
)

chain4 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step4,
    output_key="ranked_solutions"
)

C:\Users\sonya\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

print(
        overall_chain(
            {
                "input": "human colonization of Mars",
                "perfect_factors": "The distance between Earth and Mars is very large, making regular resupply difficult"
            }
    )
)

C:\Users\sonya\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.
{'input': 'human colonization of Mars', 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult', 'ranked_solutions': 'Ranking of solutions in order of promise:\n\n1. Establishing a system of regular cargo missions to Mars\n2. Deepening the thought process for developing sustainable and self-sufficient habitats on Mars\n3. Implementing a system of resource recycling and reuse on Mars\n\nJustification for ranking:\n\n1. Regular cargo missions to Mars have the potential to provide essential supplies and resources for future missions and colonization efforts. By establishing a reliable system for transporting cargo to Mars, we can ensure the success and sustainability of human presence on the planet.\n\n2. Developing sustainable habitats on Mars is crucial for long-term colonization and survival. By deepening the thought process and considering advanced materials, energy sources, 

# ReAct prompt overview

Let's review ReAct prompt as it's defined in Langchain.

In [7]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

# Self-ask with search
Let's review self-ask with search as it's defined in Langchain.

In [8]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")
prompt.template

'Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: Who was the maternal grandfather of George Washington?\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the fathe

## Memory as a retrieval tool

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

In [13]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "neurons_lab_search",
    "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!",
)

# Langchain tools

In [14]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

In [15]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [16]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

C:\Users\sonya\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [17]:
# run the agent
agent_chain.run(
    "What are the Neurons Lab services",
)

C:\Users\sonya\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I should use the Neurons Lab search tool to find information about the services they offer.
Action:
```
{
  "action": "neurons_lab_search",
  "action_input": "services"
}
```
Observation: Grow your business & attract clients 
Do you also seek development opportunities or funding? Thanks to our advanced-tier AWS partnership and access to a vast VC network, we can aid your company in obtaining grants and other assistance.

















Talk to our expert team from Europe
Get in touch








Talk to our expert team from USA
Get in touch





Articles = mass of insights /
volume of experience

ALL POSTINGS

                                    1 — 6                                    











AI for portfolio management: from Markowitz to Reinforcement Learning

                                                    Jul 16, 2023|11 min read                                                


FinTech









Accelerate Remote Patient Monito

'The Neurons Lab offers a range of services including AI feasibility analysis, AI solution engineering, AI operations management, Generative AI Solutions, GenAI Workshop + PoC, and Co-labs. They specialize in collaborating with companies in various industries such as HealthTech, CleanTech, and Retail. If you are interested in their services, you can get in touch with them for more information or to discuss potential collaborations. \n'

# Custom tools and reasoning
Here is a full guide on how to create custom tools for LangChain

In [18]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [19]:
@tool
def calculate_length_tool(a: str) -> int:
    """The function calculates the length of the input string."""
    return len(a)

In [20]:
calculate_length_tool.description, calculate_length_tool.args,

('The function calculates the length of the input string.',
 {'a': {'title': 'A', 'type': 'string'}})

In [21]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool, calculate_length_tool],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [22]:
# run the agent
agent_chain.run(
    "Find the description of Neurons Lab services and calculate the length of this description",
)



> Entering new AgentExecutor chain...
I should use the neurons_lab_search tool to find the description of Neurons Lab services and then use the calculate_length_tool to determine the length of the description.
Action: neurons_lab_search
Action Input: Neurons Lab services description
Observation: Neurons Lab | Home


























 



Your vision
Our expertise
Secret sauce*


                            Recipe                            



Partner with us


                                for INNOVATION                                                    
We are AI solution development experts for fast-growing companies seeking innovation through collaboration.






















HealthTech
CleanTech
RetailTech





AWS Certified Advanced partner
and Generative AI competency holder






AI2 = Accelerators2 + 
Handbook2


AI feasibility analysis
Discover


AI solution engineering
Launch


AI operations management
Scale





Success =(Execution x Talent) / 
(Uncertainty x

'32'

In [23]:
@tool
def calculate_uppercase_tool(a: str) -> int:
    """The function calculates the number of uppercase characters in the input string."""
    return sum(1 for c in a if c.isupper())

# Memory

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

## Long-term memory

In [25]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "neurons_lab_search",
    "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!",
)

## Short-term memory

In [26]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [27]:
message_history = ChatMessageHistory()

## Sensory memory

In [28]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import hub

In [30]:
prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")

In [31]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace

# Agent
## All tools together

In [32]:
tools = [retriever_tool, tavily_tool, calculate_length_tool, calculate_uppercase_tool]
for t in tools:
    print(t.name, t.description)

neurons_lab_search Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!
tavily_search_results_json A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
calculate_length_tool The function calculates the length of the input string.
calculate_uppercase_tool The function calculates the number of uppercase characters in the input string.


## Defining an agent with tools and memory

In [33]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [34]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

## Running an agent

In [35]:
agent_with_chat_history.invoke(
    {
        "input": "Hello!"
    },
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'Hello!',
 'chat_history': [],
 'output': 'Hello! How can I assist you today?'}

In [36]:
agent_with_chat_history.invoke(
    {
        "input": "Find the description of Neurons Lab services and calculate the length of this description"
    }, 
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `neurons_lab_search` with `{'query': 'Neurons Lab services description'}`


Neurons Lab | Home


























 



Your vision
Our expertise
Secret sauce*


                            Recipe                            



Partner with us


                                for INNOVATION                                                    
We are AI solution development experts for fast-growing companies seeking innovation through collaboration.






















HealthTech
CleanTech
RetailTech





AWS Certified Advanced partner
and Generative AI competency holder






AI2 = Accelerators2 + 
Handbook2


AI feasibility analysis
Discover


AI solution engineering
Launch


AI operations management
Scale





Success =(Execution x Talent) / 
(Uncertainty x Time)

All  stories

                                    1 — 6                                    











Creative Practice Solutions : Developing an AI-Driven Medical

{'input': 'Find the description of Neurons Lab services and calculate the length of this description',
 'chat_history': [HumanMessage(content='Hello!'),
  AIMessage(content='Hello! How can I assist you today?')],
 'output': 'The description of Neurons Lab services is 783 characters long. If you need more information or assistance, feel free to ask!'}

In [37]:
agent_with_chat_history.invoke(
    {
        "input": "Find the description of Neurons Lab services and calculate the uppercase characters of this description"
    }, 
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `neurons_lab_search` with `{'query': 'Neurons Lab services description'}`


Neurons Lab | Home


























 



Your vision
Our expertise
Secret sauce*


                            Recipe                            



Partner with us


                                for INNOVATION                                                    
We are AI solution development experts for fast-growing companies seeking innovation through collaboration.






















HealthTech
CleanTech
RetailTech





AWS Certified Advanced partner
and Generative AI competency holder






AI2 = Accelerators2 + 
Handbook2


AI feasibility analysis
Discover


AI solution engineering
Launch


AI operations management
Scale





Success =(Execution x Talent) / 
(Uncertainty x Time)

All  stories

                                    1 — 6                                    











Creative Practice Solutions : Developing an AI-Driven Medical

{'input': 'Find the description of Neurons Lab services and calculate the uppercase characters of this description',
 'chat_history': [HumanMessage(content='Hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='Find the description of Neurons Lab services and calculate the length of this description'),
  AIMessage(content='The description of Neurons Lab services is 783 characters long. If you need more information or assistance, feel free to ask!')],
 'output': 'The description of Neurons Lab services contains 61 uppercase characters. \n\nIf you have any more questions or need further assistance, feel free to ask!'}

In [38]:
agent_with_chat_history.invoke(
    {
        "input": "What is the weather today in Madrid and calculate the uppercase characters of this description"
    }, 
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather today in Madrid'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Madrid', 'region': 'Madrid', 'country': 'Spain', 'lat': 40.4, 'lon': -3.68, 'tz_id': 'Europe/Madrid', 'localtime_epoch': 1717140971, 'localtime': '2024-05-31 9:36'}, 'current': {'last_updated_epoch': 1717140600, 'last_updated': '2024-05-31 09:30', 'temp_c': 15.0, 'temp_f': 59.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 17.4, 'wind_kph': 28.1, 'wind_degree': 40, 'wind_dir': 'NE', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 44, 'cloud': 0, 'feelslike_c': 13.5, 'feelslike_f': 56.4, 'windchill_c': 22.0, 'windchill_f': 71.5, 'heatindex_c': 22.3, 'heatindex_f': 72.2, 'dewpoint_c': -0.7, 'dewpoint_f': 30.8, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph

{'input': 'What is the weather today in Madrid and calculate the uppercase characters of this description',
 'chat_history': [HumanMessage(content='Hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='Find the description of Neurons Lab services and calculate the length of this description'),
  AIMessage(content='The description of Neurons Lab services is 783 characters long. If you need more information or assistance, feel free to ask!'),
  HumanMessage(content='Find the description of Neurons Lab services and calculate the uppercase characters of this description'),
  AIMessage(content='The description of Neurons Lab services contains 61 uppercase characters. \n\nIf you have any more questions or need further assistance, feel free to ask!')],
 'output': 'The weather in Madrid today is sunny with a temperature of 15°C. \n\nThe description of the weather contains 3 uppercase characters. \n\nIf you have any more questions or need further assistanc

In [39]:
agent_with_chat_history.invoke(
    {
        "input": "Make a quick summary of what we have discussed today"
    }, 
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Today, we discussed the description of Neurons Lab services, calculated the length and uppercase characters of the description. We also looked up the weather in Madrid and calculated the uppercase characters in the weather description. 

If you have any more questions or need further assistance, feel free to let me know!

> Finished chain.


{'input': 'Make a quick summary of what we have discussed today',
 'chat_history': [HumanMessage(content='Hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='Find the description of Neurons Lab services and calculate the length of this description'),
  AIMessage(content='The description of Neurons Lab services is 783 characters long. If you need more information or assistance, feel free to ask!'),
  HumanMessage(content='Find the description of Neurons Lab services and calculate the uppercase characters of this description'),
  AIMessage(content='The description of Neurons Lab services contains 61 uppercase characters. \n\nIf you have any more questions or need further assistance, feel free to ask!'),
  HumanMessage(content='What is the weather today in Madrid and calculate the uppercase characters of this description'),
  AIMessage(content='The weather in Madrid today is sunny with a temperature of 15°C. \n\nThe description of the weather conta

## Structuring outputs

In [40]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [41]:
# Define your desired data structure.
class DesiredStructure(BaseModel):
    question: str = Field(description="the question asked")
    numerical_answer: int = Field(description="the number extracted from the answer, text excluded")
    text_answer: str = Field(description="the text part of the answer, numbers excluded")
parser = PydanticOutputParser(pydantic_object=DesiredStructure)

In [42]:
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_and_model = prompt | model
output = prompt_and_model.invoke({
    "query": "Find the description of Neurons Lab services and calculate the length of this description"}
)

In [43]:
output

AIMessage(content='{\n  "question": "Find the description of Neurons Lab services and calculate the length of this description",\n  "numerical_answer": 56,\n  "text_answer": "Neurons Lab offers a wide range of services related to artificial intelligence, machine learning, and data analysis. The description is 224 characters long."\n}', response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 253, 'total_tokens': 320}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-be5bd9c5-eb7d-4fa3-9364-2b800da2d7b2-0', usage_metadata={'input_tokens': 253, 'output_tokens': 67, 'total_tokens': 320})